In [1]:
import pandas as pd
from geopy.distance import geodesic
import numpy as np

# ============================================
# 1. Cargar datasets
# ============================================
df_clima = pd.read_csv("/content/sample_data/Biobio_2024_Normalizado.csv")
df_fire = pd.read_csv("/content/sample_data/Incendios_Chile.csv")

# Asegurar formato de fecha compatible
df_clima["date"] = pd.to_datetime(df_clima["date"]).dt.date
df_fire["date"] = pd.to_datetime(df_fire["acq_date"]).dt.date

# ============================================
# 2. Extraer coordenadas representativas por comuna
# ============================================
coords_comunas = df_clima[["comuna", "latitud", "longitud"]].drop_duplicates()

# ============================================
# 3. Función para asignar comuna al incendio por distancia mínima
# ============================================
def asignar_comuna(lat, lon):
    distancia_min = 999999
    comuna_sel = None

    for _, row in coords_comunas.iterrows():
        dist = geodesic((lat, lon), (row["latitud"], row["longitud"])).km
        if dist < distancia_min:
            distancia_min = dist
            comuna_sel = row["comuna"]

    return comuna_sel

# Aplicar a todos los incendios
df_fire["comuna"] = df_fire.apply(
    lambda x: asignar_comuna(x["latitude"], x["longitude"]),
    axis=1
)

# ============================================
# 4. Unir incendios con clima (1 a 1, preservando cada incendio)
# ============================================
df_merge = df_fire.merge(
    df_clima,
    on=["date", "comuna"],
    how="left"
)

# ============================================
# 5. Crear registros para días sin incendios
# ============================================
df_comb = df_clima.merge(
    df_fire[["date", "comuna"]],
    on=["date", "comuna"],
    how="left",
    indicator=True
)

df_sin_incendios = df_comb[df_comb["_merge"] == "left_only"].drop(columns=["_merge"])

# Agregar columnas de incendios vacías
cols_incendios = [
    "latitude", "longitude", "bright_ti4", "bright_ti5", "scan", "track",
    "acq_time", "satellite", "confidence", "version", "frp", "daynight", "type"
]

for c in cols_incendios:
    df_sin_incendios[c] = None

# ============================================
# 6. Combinar ambos: con incendios y sin incendios
# ============================================
df_final = pd.concat([df_merge, df_sin_incendios], ignore_index=True)

# ============================================
# 7. Calcular distancia incendio ↔ centro de comuna
# ============================================

coord_dict = {
    row["comuna"]: (row["latitud"], row["longitud"])
    for _, row in coords_comunas.iterrows()
}

def calcular_distancia(row):
    if pd.isna(row["latitude"]) or pd.isna(row["longitude"]):
        return np.nan
    lat_inc, lon_inc = row["latitude"], row["longitude"]
    lat_c, lon_c = coord_dict[row["comuna"]]
    return geodesic((lat_inc, lon_inc), (lat_c, lon_c)).km

df_final["distancia_km"] = df_final.apply(calcular_distancia, axis=1)

# ============================================
# 8. Clasificación del incendio según FRP
# ============================================

def clasificar_frp(frp):
    if pd.isna(frp):
        return "sin_incendio"
    if frp < 3:
        return "bajo"
    elif frp < 10:
        return "medio"
    else:
        return "alto"

df_final["categoria_incendio"] = df_final["frp"].apply(clasificar_frp)

# ============================================
# 9. Variables binarias
# ============================================

# ¿Hubo incendio?
df_final["incendio"] = df_final["frp"].apply(lambda x: 0 if pd.isna(x) else 1)

# Dummies según categoría
df_final["incendio_bajo"] = (df_final["categoria_incendio"] == "bajo").astype(int)
df_final["incendio_medio"] = (df_final["categoria_incendio"] == "medio").astype(int)
df_final["incendio_alto"] = (df_final["categoria_incendio"] == "alto").astype(int)

# Día/noche
df_final["es_noche"] = df_final["daynight"].apply(lambda x: 1 if x == "N" else 0)

# ============================================
# 10. Ordenar columnas
# ============================================

cols_incendios_ext = [
    "latitude", "longitude", "bright_ti4", "bright_ti5", "scan", "track",
    "acq_time", "satellite", "confidence", "version", "frp", "daynight", "type",
    "distancia_km", "categoria_incendio", "incendio",
    "incendio_bajo", "incendio_medio", "incendio_alto", "es_noche"
]

columnas_finales = (
    [c for c in df_final.columns if c not in cols_incendios_ext] +
    cols_incendios_ext
)

df_final = df_final[columnas_finales]

# ============================================
# 11. Exportar archivo final
# ============================================
df_final.to_csv("Clima_Incendios_Detallado_Clasificado.csv",
                index=False, encoding="utf-8-sig")

print("🎉 Archivo final generado con éxito")
print("Filas totales:", len(df_final))


/tmp/ipython-input-1352552209.py:74: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_merge, df_sin_incendios], ignore_index=True)


🎉 Archivo final generado con éxito
Filas totales: 187891


## Resumen final del proceso

Este notebook documenta el proceso metodológico de integración entre datos de focos de incendio y variables climáticas asociadas. Mediante etapas sucesivas de carga, normalización temporal, depuración, integración espacio-temporal y validación, se construye un dataset coherente que vincula incendios con su contexto ambiental.

El resultado constituye un insumo robusto y reproducible para análisis exploratorios, modelos predictivos y sistemas de apoyo a la toma de decisiones en la gestión del riesgo de incendios forestales.